In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, LinearRegression
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler

RSEED = 42

In [2]:
df_train_prepro = pd.read_csv('data/preprocessed_train_data_with_date_hol_concat.csv')

In [3]:
df_train_prepro

,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,target,duration,dep_hour,dep_day,...,dep_temp,dep_precip,dep_wind,arr_temp,arr_precip,arr_wind,Country,City,is_holiday,holiday_length
0,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,-19.0,9300.0,4,16,...,9.4,9.9,23.6,12.5,0.0,16.5,Tunisia,Tunis,0,0
1,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,-48.0,10200.0,14,17,...,11.7,0.0,41.2,2.7,2.6,8.8,Tunisia,Djerba,0,0
2,TUN,MED,2016-01-20 19:40:00,2016-01-21 00:00:00,ATA,TU 320IMR,-16.0,15600.0,19,20,...,11.1,1.3,6.8,22.3,23.9,11.8,Tunisia,Tunis,0,0
3,IST,TUN,2016-01-21 20:10:00,2016-01-21 23:00:00,ATA,TU 320IMU,-8.0,10200.0,20,21,...,5.2,0.0,10.8,10.3,0.0,5.0,Turkey,Istanbul,0,0
4,CMN,TUN,2016-01-22 17:45:00,2016-01-22 20:10:00,ATA,TU 320IMR,-37.0,8700.0,17,22,...,16.0,0.0,9.8,10.6,2.0,4.6,Morocco,Casablanca,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99742,TUN,DJE,2018-04-18 08:20:00,2018-04-18 09:10:00,ATA,TU 31BIMQ,8.0,3000.0,8,18,...,16.8,0.0,14.2,17.1,0.0,20.2,Tunisia,Tunis,0,0
99743,ORY,DJE,2018-12-05 10:15:00,2018-12-05 13:05:00,ATA,TU 736IOL,20.0,10200.0,10,5,...,9.6,1.2,10.7,17.3,0.0,12.5,France,Paris,0,0
99744,BRU,DJE,2018-12-05 09:45:00,2018-12-05 12:50:00,ATA,TU 736IOR,15.0,11100.0,9,5,...,19.1,0.0,12.4,17.3,0.0,12.5,Belgium,Brussels,0,0
99745,ORY,TUN,2018-12-04 18:30:00,2018-12-04 20:55:00,ATA,TU 32AIMI,22.0,8700.0,18,4,...,10.2,1.2,8.2,16.7,0.0,19.6,France,Paris,0,0


In [5]:
y = df_train_prepro['target']
X = df_train_prepro.drop(columns=['target'])

In [ ]:
# Define the categorical features
num_col = ['duration','dep_temp', 'dep_precip', 'dep_wind', 'arr_temp',
       'arr_precip', 'arr_wind', 'holiday_length']
cat_col = ['departure_point', 'arrival_point', 'flight_status', 'aircraft_code','dep_hour',
       'dep_day', 'dep_month', 'dep_dayofweek', 'dep_quarter', 'dep_season',
       'dep_is_weekend', 'dep_time_of_day', 'arr_hour', 'arr_day', 'arr_month',
       'arr_dayofweek', 'arr_quarter', 'arr_season', 'arr_is_weekend',
       'arr_time_of_day', 'route', 'is_holiday', 'Country', 'City']

In [9]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Use sparse output for OneHotEncoder to save memory
encoder = OneHotEncoder(sparse=True, handle_unknown='ignore')
scaler = StandardScaler()

# Fit and transform categorical columns (sparse matrix)
X_cat_sparse = encoder.fit_transform(X[cat_col])

# Scale only the numerical columns and convert to float32
X_num_scaled = scaler.fit_transform(X[num_col]).astype(np.float32)

# Convert sparse matrix to float32 and combine with numerical features
from scipy import sparse
X_encoded_scaled = sparse.hstack([X_num_scaled, X_cat_sparse.astype(np.float32)]).tocsr()

# Split the encoded and scaled data
X_train_1, X_train_2, y_train_1, y_train_2 = train_test_split(
    X_encoded_scaled, y, stratify=y, test_size=0.2, random_state=RSEED
)

c:\Users\marco\Desktop\WaterMLops\ds-ml-project\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [10]:
X_encoded_scaled.shape

(99747, 1397)

XGB RandomisedSearchCV

Find the best parameters for XGBRegressor

In [ ]:
# Fit model to training data
xgb_best = XGBRegressor(objective='reg:squarederror', random_state=42)

In [ ]:
#hyperparameter grid
xgb_param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [3, 4, 5, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8],
    'colsample_bytree': [0.6, 0.8],
    'gamma': [0, 0.1, 0.2, 0.5],
    'min_child_weight': [1, 3, 5],
    'scale_pos_weight': [1, 2]
}

In [ ]:
xgb_search = GridSearchCV(XGBRegressor(objective='reg:squarederror', random_state=42), xgb_param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
xgb_search.fit(X_train_1, y_train_1)
best_xgb = xgb_search.best_estimator_

In [15]:
#hyperparameter grid
xgb_param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [3, 4, 5, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.5, 1],
    'min_child_weight': [1, 3, 5, 7],
    'scale_pos_weight': [1, 2, 3]
}

xgb_ransearcv_best = RandomizedSearchCV(estimator=xgb_best, param_distributions=xgb_param_grid, 
                          n_iter=100, scoring='accuracy', cv=3, 
                          verbose=1, random_state=42, n_jobs=-1)
xgb_ransearcv_best.fit(X_train_1, y_train_1)

NameError: name 'xgb_best' is not defined

In [ ]:
rf_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5]
}

xgb_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

ridge_param_grid = {
    'alpha': [0.1, 1.0, 10.0]
}

lgbm_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

In [ ]:
from sklearn.model_selection import GridSearchCV

# Random Forest
rf_search = GridSearchCV(RandomForestRegressor(random_state=42), rf_param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
rf_search.fit(X_train_1, y_train_1)
best_rf = rf_search.best_estimator_

# XGBoost
xgb_search = GridSearchCV(XGBRegressor(objective='reg:squarederror', random_state=42), xgb_param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
xgb_search.fit(X_train_1, y_train_1)
best_xgb = xgb_search.best_estimator_

# Ridge
ridge_search = GridSearchCV(Ridge(random_state=42), ridge_param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
ridge_search.fit(X_train_1, y_train_1)
best_ridge = ridge_search.best_estimator_

# LightGBM
lgbm_search = GridSearchCV(LGBMRegressor(random_state=42), lgbm_param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
lgbm_search.fit(X_train_1, y_train_1)
best_lgbm = lgbm_search.best_estimator_

Create Base models + Meta model

In [12]:
# Base models
random_forest = RandomForestRegressor(random_state=42)
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)
ridge = Ridge(random_state=42)
knn = KNeighborsRegressor()
#lgbm = LGBMRegressor(random_state=42)

# Meta-model
linear_model = LinearRegression()

In [13]:
X_train_1, X_train_2, y_train_1, y_train_2 = train_test_split(X_encoded_scaled, y, stratify=y, test_size=0.2, random_state=RSEED)

In [ ]:
# Fit all the base estimators on the 1st half of the train dataset
rf_model = random_forest.fit(X_train_1, y_train_1)
xgb_model = xgb.fit(X_train_1, y_train_1)
ridge_model = ridge.fit(X_train_1, y_train_1)

# KNeighborsRegressor does not support sparse input, so convert to dense
knn_model = knn.fit(X_train_1.toarray(), y_train_1)

#lgbm_model = lgbm.fit(X_train_1, y_train_1)

MemoryError: could not allocate 1835008 bytes

In [ ]:
# Then with the second half of the train dataset we predict the values from the base estimators
rf_pred = rf_model.predict(X_train_2)
xgb_pred = xgb_model.predict(X_train_2)
ridge_pred = ridge_model.predict(X_train_2)
knn_pred = knn_model.predict(X_train_2)
#lgbm_pred = lgbm_model.predict(X_train_2)

# Combine base model predictions for meta-model input
combine_X_pred_test = pd.concat([
	pd.DataFrame(rf_pred),
	pd.DataFrame(xgb_pred),
	pd.DataFrame(ridge_pred),
    pd.DataFrame(knn_pred)
    #pd.DataFrame(lgbm_pred)
], axis=1)

In [50]:
# Fit the final estimator on the combined probabilities and target values
linear_model.fit(combine_X_pred_test, y_train_2)

LinearRegression()

In [51]:
# Predict with meta-model
y_pred = linear_model.predict(combine_X_pred_test)

In [52]:
# Ensure predictions are non-negative
y_pred[y_pred < 0] = 0
y_train_2 = y_train_2.clip(lower=0)

In [53]:
# Evaluate the model
mse = mean_squared_error(y_train_2, y_pred)
r2 = r2_score(y_train_2, y_pred)
rmse = np.sqrt(mean_squared_error(y_train_2, y_pred))
print(f'Mean Squared Error: {mse}')
print(f'R2 Score: {r2}')
print(f"Stacking RMSE: {rmse:.2f}")

Mean Squared Error: 1147.4653411269428
R2 Score: 0.14211245445344112
Stacking RMSE: 33.87


In [21]:
# Get feature importances from each base model and display them

# Random Forest feature importances
rf_importances = rf_model.feature_importances_

# XGBoost feature importances
xgb_importances = xgb_model.feature_importances_

# Ridge feature coefficients (absolute value for importance)
ridge_importances = np.abs(ridge_model.coef_)

# LightGBM feature importances
lgbm_importances = lgbm_model.feature_importances_

# Feature names
feature_names = num_col + list(encoded_cat_cols)

# Create a DataFrame for each model's importances
importances_df = pd.DataFrame({
    'feature': feature_names,
    'RandomForest': rf_importances,
    'XGBoost': xgb_importances,
    'Ridge': ridge_importances,
    'LightGBM': lgbm_importances
})

# Show top 15 features by average importance across models
importances_df['avg_importance'] = importances_df[['RandomForest', 'XGBoost', 'Ridge', 'LightGBM']].mean(axis=1)
importances_df.sort_values('avg_importance', ascending=False)

,feature,RandomForest,XGBoost,Ridge,LightGBM,avg_importance
0,duration,0.170090,0.001936,0.000296,610,152.543081
135,arrival_point_AMM,0.000306,0.000918,107.505135,0,26.876590
251,arrival_point_VNO,0.000240,0.000320,75.580911,0,18.895368
233,arrival_point_RTM,0.000445,0.001048,70.245198,0,17.561673
85,departure_point_MVB,0.000111,0.000601,67.874797,0,16.968877
...,...,...,...,...,...,...
62,departure_point_KEF,0.000000,0.000000,0.000000,0,0.000000
284,aircraft_code_OL 321ABY,0.000000,0.000000,0.000000,0,0.000000
298,aircraft_code_TU 32A32A,0.000000,0.000000,0.000000,0,0.000000
332,Country_Angola,0.000000,0.000000,0.000000,0,0.000000


In [ ]:
# Get feature importances from each base model and display them

# Random Forest feature importances
rf_importances = rf_model.feature_importances_

# XGBoost feature importances
xgb_importances = xgb_model.feature_importances_

# Ridge feature coefficients (absolute value for importance)
ridge_importances = np.abs(ridge_model.coef_)

# LightGBM feature importances
lgbm_importances = lgbm_model.feature_importances_

# Feature names
feature_names = num_col + list(encoded_cat_cols)

# Create a DataFrame for each model's importances
importances_df = pd.DataFrame({
    'feature': feature_names,
    'RandomForest': rf_importances,
    'XGBoost': xgb_importances,
    'Ridge': ridge_importances,
    'LightGBM': lgbm_importances
})

# Show top 15 features by average importance across models
importances_df['avg_importance'] = importances_df[['RandomForest', 'XGBoost', 'Ridge', 'LightGBM']].mean(axis=1)
importances_df.sort_values('avg_importance', ascending=False).head(15)

In [ ]:
# For linear_model (LinearRegression), use coef_ instead of feature_importances_
feature_names = num_col + list(encoded_cat_cols)
fi = pd.DataFrame({
    'feature': feature_names,
    'importance': linear_model.coef_.flatten()
}).sort_values('importance', ascending=False)
fi.head()

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [ ]:
meta_feature_names = ['rf_pred', 'xgb_pred', 'ridge_pred', 'lgbm_pred']
fi = pd.DataFrame({
    'feature': meta_feature_names,
    'importance': linear_model.coef_.flatten()
}).sort_values('importance', ascending=False)
fi.head()

ValueError: All arrays must be of the same length